### DATA INGESTION + VECTOR DB PIPELINE

In [15]:
import os 
from langchain_community.document_loaders import PyPDFLoader , PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path 


In [19]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error in {pdf_file.name}: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 3 PDF files to process

Processing: ITI_AgenticAI_Final.pdf
  ✓ Loaded 16 pages

Processing: JAAI-V3N2-41.pdf
  ✓ Loaded 9 pages

Processing: AI.pdf
  ✓ Loaded 30 pages

Total documents loaded: 55


In [21]:
# all_pdf_documents

In [23]:
### SPLITTING THE DOCUMENTS INTO CHUNKS ###
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )
    
    split_docs=text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    # Example output of first chunk
    if split_docs:
        print("\nExample chunk:")
        print(split_docs[0].page_content[:200])  # Print first 200 characters of the first chunk
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [25]:
chunks=split_documents(all_pdf_documents)
print(f"\nTotal chunks created: {len(chunks)}")
print(chunks[0])

Split 55 documents into 358 chunks

Example chunk:
www.itic.org
Promoting Innovation Worldwide 1
Understanding 
Agentic AI: 
ITI's Policy Guide
November 2025
Metadata: {'producer': 'macOS Version 15.6.1 (Build 24G90) Quartz PDFContext, AppendMode 1.1', 'creator': 'Adobe InDesign 20.0 (Macintosh)', 'creationdate': '2025-11-07T10:25:21-05:00', 'trapped': '/False', 'moddate': "D:20251107152735Z00'00'", 'source': '../data/PDF/ITI_AgenticAI_Final.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1', 'source_file': 'ITI_AgenticAI_Final.pdf', 'file_type': 'pdf'}

Total chunks created: 358
page_content='www.itic.org
Promoting Innovation Worldwide 1
Understanding 
Agentic AI: 
ITI's Policy Guide
November 2025' metadata={'producer': 'macOS Version 15.6.1 (Build 24G90) Quartz PDFContext, AppendMode 1.1', 'creator': 'Adobe InDesign 20.0 (Macintosh)', 'creationdate': '2025-11-07T10:25:21-05:00', 'trapped': '/False', 'moddate': "D:20251107152735Z00'00'", 'source': '../data/PDF/ITI_AgenticAI_Final.p

### EMBEDDING AND VECTOR STORE DB

In [27]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity


In [28]:

class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


### VECTOR STORE

In [ ]:
### VECTOR STORE MANAGEMENT USING CHROMADB ###
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore



Vector store initialized. Collection: all_pdf_documents
Existing documents in collection: 0


In [38]:

### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 358 texts...


Batches: 100%|██████████| 12/12 [00:04<00:00,  2.53it/s]


Generated embeddings with shape: (358, 384)
Adding 358 documents to vector store...
Successfully added 358 documents to vector store
Total documents in collection: 358


### RETRIEVER PIPELINE FROM VECTOR STORE